In [3]:
%matplotlib inline
import pandas as pd
import os
import random
import re
import nltk
import string
import urllib2
import math
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
# from gensim import corpora
# from gensim.models import LdaMulticore
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from nltk.stem import WordNetLemmatizer
from collections import defaultdict
import copy as cp
from bs4 import BeautifulSoup
from functools import reduce 
import operator
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
import datetime
from textblob import TextBlob

In [4]:
#2013-2016 emergence of populist concerns 
# Frequency of Populist key words 
#data_file = '~/Documents/Small Projects/AAPOR_2018/Populism/'
data_file = '~/Documents/personal/aapor_politics_2018/'
populist_posts = pd.read_csv(data_file + "populist_comments_sample.csv")

In [36]:
#redo with more data 
data = os.listdir("all_populist")

df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat((pd.read_csv(f,header=None,skip_rows=1) for f in data), ignore_index=True) 


In [5]:
populist_posts['year'] = populist_posts['time_created'].apply(lambda x :datetime.datetime.fromtimestamp(x).year)
populist_posts['month'] = populist_posts['time_created'].apply(lambda x :datetime.datetime.fromtimestamp(x).month)

In [4]:
#data = populist_posts #just rename to make it faster

In [37]:
train_topic_model = shuffle(populist_posts)
#[:20000]
data = train_topic_model


In [39]:
data.shape

(1397056, 11)

In [ ]:
#make a subclass of the vectorizer that lemmatizes so we don't end up with wierd truncations
# lemm = WordNetLemmatizer()
# class LemmaCountVectorizer(CountVectorizer):
#     def build_analyzer(self):
#         analyzer = super(LemmaCountVectorizer, self).build_analyzer()
#         return lambda doc: (lemm.lemmatize(w) for w in analyzer(doc))

In [40]:
all_text = list(data['body'])

### Get the Top Words used by Subreddit and Time

In [41]:
def get_top_words(text):
    tf_vectorizer = CountVectorizer(max_df=0.95, 
                                         min_df=len(text)/100,
                                         stop_words='english',
                                         decode_error='ignore')
    tf = tf_vectorizer.fit_transform(text)
    feature_names = tf_vectorizer.get_feature_names()
    count_vec = np.asarray(tf.sum(axis=0)).ravel()
    zipped = list(zip(feature_names, count_vec))
    words, freqs = (list(x) for x in zip(*sorted(zipped, key=lambda x: x[1], reverse=True)))
    words = words[:40]
    freqs = freqs[:40]
    return words
#     top_words = pd.DataFrame([words, freqs]).T
#     top_words

In [42]:
#the top words
get_top_words(list(data['body']))

[u'trump',
 u'people',
 u'gt',
 u'just',
 u'like',
 u'don',
 u'clinton',
 u'white',
 u'think',
 u'com',
 u'www',
 u'right',
 u'http',
 u'know',
 u'hillary',
 u'going',
 u'said',
 u'sanders',
 u'https',
 u'want',
 u'make',
 u'time',
 u'did',
 u'say',
 u'politics',
 u'really',
 u'way',
 u'doesn',
 u'vote',
 u'president',
 u'actually',
 u'good',
 u'country',
 u'does',
 u'obama',
 u'isn',
 u'didn',
 u'news',
 u'point',
 u'thing']

In [43]:
list_of_subreddits = ['worldnews','news','history','politics', 'democrats'] #these are the largest
top_words_grid = []
cols = []
for subreddit in list_of_subreddits:
    cols.append(subreddit)
    top_words_grid.append( get_top_words(list(data[data['subreddit']==subreddit]['body'])))
    print subreddit
top_words_df = pd.DataFrame(top_words_grid).T
top_words_df.columns = cols
top_words_df

worldnews
news
history
politics
democrats


,worldnews,news,history,politics,democrats
0,people,people,war,trump,trump
1,trump,white,people,people,sanders
2,gt,trump,white,clinton,clinton
3,just,just,history,gt,people
4,like,like,like,just,gt
5,don,gt,time,like,like
6,islam,don,just,don,don
7,think,black,gt,think,just
8,http,think,did,com,hillary
9,com,right,com,hillary,bernie


In [44]:
top_words_grid_yr = []
year_list = []
for year in [2013, 2014, 2015, 2016, 2017]:
    year_list.append(year)
    top_words_grid_yr.append(get_top_words(list(data[data['year']==year]['body'])))
    print year
top_words_df_year = pd.DataFrame(top_words_grid_yr).T
top_words_df_year.columns = year_list
top_words_df_year.head(15)

2013
2014
2015
2016
2017


,2013,2014,2015,2016,2017
0,people,people,people,trump,trump
1,white,gt,gt,people,people
2,gt,white,just,clinton,gt
3,just,just,like,gt,just
4,like,like,white,just,like
5,don,don,don,like,don
6,http,http,think,don,white
7,think,think,trump,hillary,think
8,com,www,sanders,think,com
9,www,com,com,sanders,right


### Topic Modelling

In [45]:
tf_vectorizer = CountVectorizer(max_df=0.95, 
                                         min_df=len(all_text)/1000,
                                         stop_words='english',
                                         decode_error='ignore')
transformed_text = tf_vectorizer.fit_transform(all_text)

In [46]:
topic_model = LatentDirichletAllocation(n_components=15, max_iter=5,
                                learning_method = 'online',
                                random_state = 0)

topic_model.fit(transformed_text)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=15, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [47]:
feature_names = tf_vectorizer.get_feature_names()
for topic in topic_model.components_:
    word_ixes = topic.argsort()[:-10:-1]
    words = [feature_names[i] for i in word_ixes]
    print words
    print "______________________________"
#labels 
labels = ["Obama","Brexit","Trump: Scandals","Sanders","World Politics","Partisan Politics","Islam","Immigration",
          "Trump: Presidency","Colloquial speach","Clinton","Russia","Race","Internet","Terrorism"]

[u'https', u'twitter', u'current', u'politics', u'com', u'rules', u'original', u'np', u'bot']
______________________________
[u'gt', u'news', u'said', u'article', u'israel', u'anti', u'read', u'jews', u'fox']
______________________________
[u'obama', u'party', u'got', u'case', u'bush', u'votes', u'new', u'years', u'congress']
______________________________
[u'white', u'people', u'black', u'racist', u'seen', u'police', u'crime', u'violence', u'men']
______________________________
[u'politics', u'reddit', u'amp', u'com', u'https', u'www', u'comments', u'message', u'removal']
______________________________
[u'islam', u'war', u'world', u'muslims', u'muslim', u'islamic', u'country', u'countries', u'middle']
______________________________
[u'right', u'maybe', u'media', u'100', u'fake', u'wing', u'trumps', u'unless', u'speech']
______________________________
[u'immigration', u'government', u'money', u'illegal', u'tax', u'years', u'country', u'work', u'job']
______________________________
[u'p

In [48]:
def which_topic(topic_model, text):
    '''
    Helper function to return a random topic if the best match is tied
    '''
    x = topic_model.transform(tf_vectorizer.transform([text]))[0]
    max_value = x[x.argsort()[-1]]
    possible_topics = []
    for index, value in enumerate(x):
        if value == max_value:
            possible_topics.append(index)
    return np.random.choice(possible_topics, 1)[0]

In [49]:
data['topic'] = data['body'].apply(lambda x: which_topic(topic_model,x))

/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [50]:
data.head()

,body,downs,score,time_created,author,subreddit,subreddit_id,month_created,selector,year,month,topic
220869,"Yeah, this is basically how I view it. I think...",NaN,2.0,1475102539,Cuckberg,politics,t5_2cneq,1,3,2016,9,6
539983,&gt;If Trump cuts taxes by every Hillary suppo...,NaN,NaN,1478911937,mrrp,politics,t5_2cneq,1,3,2016,11,7
213255,"Urm no, it's a perfectly legit question, someo...",NaN,5.0,1475066137,Quagers,ukpolitics,t5_2qhcv,1,3,2016,9,8
260436,The point is that he DID say something. And in...,NaN,NaN,1502738136,kylethemurphy,politics,t5_2cneq,1,3,2017,8,8
312474,Is there anything Sanders HASN'T promised yet?...,0.0,2.0,1431695035,Jura52,politics,t5_2cneq,1,3,2015,5,1


### Sentiment

In [52]:
#try excepts are to skip unicode errors which I don't want to deal with
#Somewhat slow on partial data (~30 sec)
def get_polarity(text):
    try:
        return TextBlob(text).sentiment.polarityt
    except:
        return np.nan
def get_subjectivity(text):
    try:
        return TextBlob(text).sentiment.subjectivity
    except:
        return np.nan
data['polarity'] = data['body'].apply(lambda x: get_polarity(x))
data['subjectivity'] = data['body'].apply(lambda x: get_subjectivity(x))

/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


### Clean Up and Analysis

In [51]:
#topic balance over time, I'm imagining one of those area plots
topics_over_time = data.groupby(['year', 'month', 'topic']).agg({'body': 'count'}).reset_index().rename(columns = {'body':'count'})
topics_over_time.to_csv("topics_over_time.csv")

In [53]:
sentiment_over_time = data.groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean})
#use this to make a  line graph, show that polarity isn't hugely changing ovet time neither is subjectivity
sentiment_over_time.to_csv("sentiment_over_times.csv")

In [54]:
sentiment_by_topic = data.groupby('topic').agg({'subjectivity': np.mean, 'polarity': np.mean, })
#also not big differences by topic
sentiment_by_topic.head()

,polarity,subjectivity
topic,,
0,0.139685,0.387926
1,0.041612,0.382984
2,0.050321,0.335446
3,0.008691,0.354988
4,0.103581,0.359253


In [86]:
#find very negative tweet
min(data['subjectivity'])
data[data['subjectivity']==1].head(15)

,body,downs,score,time_created,author,subreddit,subreddit_id,month_created,selector,year,month,topic,polarity,subjectivity,Clinton,Trump,altright
1364757,&gt;charlatan\n\nis the perfect word to descri...,NaN,NaN,1485112032,II-III-V-VII-XI,politics,t5_2cneq,1,3,2017,1,1,1.0000,1.0,False,True,False
456682,https://en.wikipedia.org/wiki/Narcissistic_per...,NaN,3.0,1462978137,StonerMeditation,politics,t5_2cneq,1,3,2016,5,13,1.0000,1.0,False,True,False
1290375,&gt; inept and corrupt\n\naka Donald Trump.\n\n,NaN,NaN,1495756535,busmans,politics,t5_2cneq,1,3,2017,5,8,-0.5000,1.0,False,True,False
600770,"NK are brilliant, they had to know the time of...",NaN,NaN,1487053137,branager,politics,t5_2cneq,1,3,2017,2,14,0.9000,1.0,False,True,False
1211070,Evidence: [1](https://www.theguardian.com/poli...,NaN,NaN,1498209331,rumdiary,ukpolitics,t5_2qhcv,1,3,2017,6,9,-0.7500,1.0,False,False,False
1014465,"Careful. Math like that, Trump might try and h...",NaN,NaN,1489691332,MisfortunateFox,politics,t5_2cneq,1,3,2017,3,14,-0.1000,1.0,False,True,False
298102,The Trump supports will still argue against th...,NaN,0.0,1449709936,Grudgecrown,politics,t5_2cneq,1,3,2015,12,10,-0.5000,1.0,False,True,False
963494,"""In honor of all of the victims of the very sa...",NaN,NaN,1506950738,Sessions4Prison,politics,t5_2cneq,1,3,2017,10,6,-0.8125,1.0,False,True,False
599574,Will Trump turn into a kaiju on November 9th a...,NaN,NaN,1476289235,Tarlcabot18,politics,t5_2cneq,1,3,2016,10,12,-0.5000,1.0,False,True,False
53162,"Trump Supporters:""You cant trust the media"".\n...",NaN,NaN,1476911833,mwinks99,politics,t5_2cneq,1,3,2016,10,11,-1.0000,1.0,False,True,False


In [88]:
data['body'].ix[1144987]

"Oh dear god. They're going to crucify Clinton to unite their base while solidifying  power."

In [55]:
data['Clinton'] = data['body'].apply(lambda x : True if 'clinton' in x.lower() or 'hilary' in x.lower() else False)
data['Trump'] = data['body'].apply(lambda x : True if 'trump' in x.lower() or 'donald' in x.lower() else False)

/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [58]:
data['altright'] = data['body'].apply(lambda x : True if 'alt-right' in x.lower() or 'alt right' in x.lower() else False)

/Users/ckelley/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
clinton_sentiment = np.mean(data[data['Clinton']]['polarity'])
trump_sentiment = np.mean(data[data['Trump']]['polarity'])
clinton_sentiment, trump_sentiment

(0.06350303471948061, 0.04926718134116553)

In [57]:
clinton_over_time = data[data['Clinton']].groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean, 'body': 'count'}).rename(columns= {'body': 'post_count'})
clinton_over_time.to_csv("clinton_polarity.csv")

In [59]:
trump_over_time = data[data['Trump']].groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean, 'body': 'count'}).rename(columns= {'body': 'post_count'})
trump_over_time.to_csv("trump_polarity.csv")

In [60]:
alt_over_time = data[data['altright']].groupby(['year', 'month']).agg({'polarity': np.mean, 'subjectivity': np.mean, 'body': 'count'}).rename(columns= {'body': 'post_count'})
alt_over_time.to_csv("altright_polarity.csv")

In [29]:
data.groupby('subreddit')['body'].count() #not enough in a lot of subreddits for much divided analysis

subreddit
business               14
democrats              48
education               2
energy                  5
environment            22
feminisms               6
history                70
law                    26
moderatepolitics       16
news                 2369
politics            13012
progressive            24
racism                  8
socialism             113
ukpolitics            685
uspolitics             10
worldevents             5
worldnews            3457
worldpolitics         108
Name: body, dtype: int64

In [ ]:
# (optional) Word Embedding

In [ ]:
# Sentiment Analysis 